# Objaverse

Objaverse is a massive dataset of annotated 3D objects.

It is hosted on 🤗[Hugging Face](https://huggingface.co/datasets/allenai/objaverse). 

This is a sample demo for the blog about Generative AI: https://ai-illustwitter.art 

In [1]:
!pip install --upgrade --quiet objaverse

import objaverse
objaverse.__version__

'0.0.7'

# Loading UIDs

Each object has a unique corresponding UID (universal identifier). Let's get all the object UIDs:

In [2]:
uids = objaverse.load_uids()
len(uids), type(uids)

(798759, list)

Get a specific UID (phoenix) that will later be shown in Augmented Realilty using AI Illustwitter

In [16]:
#Flying phoenix: https://sketchfab.com/3d-models/phoenix-bird-844ba0cf144a413ea92c779f18912042
filter_object = filter(lambda a: '844ba0cf144a413ea92c779f18912042' in a, uids)

In [17]:
filter_object_list = list(filter_object);
print(filter_object_list)


['844ba0cf144a413ea92c779f18912042']


Here, `uids` is a list of strings. In this example, we have only 1 UID

# Loading Annotations

We can get the object annotations for each of object using the
```python
objaverse.load_annotations(
    uids: Optional[List[str]] = None
) -> Dict[str, Dict[str, Any]]
```

function. The function optionally takes in a list of the `uids` and returns a map of each specified uid to its corresponding annotations. If `uids` is not specified, the annotations for every object is returned.

In [18]:
annotations = objaverse.load_annotations(filter_object_list[:1])
annotations

{'844ba0cf144a413ea92c779f18912042': {'uri': 'https://api.sketchfab.com/v3/models/844ba0cf144a413ea92c779f18912042',
  'uid': '844ba0cf144a413ea92c779f18912042',
  'name': 'phoenix bird',
  'staffpickedAt': '2018-06-04T08:22:14.718170',
  'viewCount': 103818,
  'likeCount': 2864,
  'animationCount': 1,
  'viewerUrl': 'https://sketchfab.com/3d-models/844ba0cf144a413ea92c779f18912042',
  'embedUrl': 'https://sketchfab.com/models/844ba0cf144a413ea92c779f18912042/embed',
  'commentCount': 201,
  'isDownloadable': True,
  'publishedAt': '2018-01-14T03:55:44.716224',
  'tags': [{'name': 'flying',
    'slug': 'flying',
    'uri': 'https://api.sketchfab.com/v3/tags/flying'},
   {'name': 'bird',
    'slug': 'bird',
    'uri': 'https://api.sketchfab.com/v3/tags/bird'},
   {'name': 'phoenix',
    'slug': 'phoenix',
    'uri': 'https://api.sketchfab.com/v3/tags/phoenix'},
   {'name': 'dragon',
    'slug': 'dragon',
    'uri': 'https://api.sketchfab.com/v3/tags/dragon'}],
  'categories': [{'name': 

Here, we see properties like the "name", "license", "description", "tags", and other metadata.

### Filtering

We can use the annotations to filter for particular properties of objects. For example, we can filter for objects that are distributed with the CC-BY license:

In [19]:
annotations = objaverse.load_annotations()

100%|██████████| 160/160 [02:05<00:00,  1.28it/s]


> Note: Loading all annotations for the first time may take a few minutes. The annotations are being downloaded and saved locally.

And then use a list comprehension to get the CC-BY licensed objects

In [20]:
cc_by_uids = [uid for uid, annotation in annotations.items() if annotation["license"] == "by"]
cc_by_uids[:10]

['94db219c315742909fee67deeeacae15',
 'fc1339e225b7408caec82681be2746c5',
 '7b56f2bdfd2845588f6bde0c5362fd0d',
 '11102f046e7846b8b4053bce5779d95c',
 '963dca3a0a7b4d6caacab65165829470',
 'e406fd52136949a688d2ed5879361021',
 'f713688895dc49c7ac3d5544ec3f8387',
 'a0abc7880573486c89340e5b52b398a0',
 '898e9be8c8964f6298188a2901216e80',
 '4068b76c755349edbfac42eaa4ea34e5']

# Downloading Objects

We're going to use multiprocessing to download the objects. First, let's check our CPU count to get the number of processes we'll use:

In [21]:
import multiprocessing
processes = multiprocessing.cpu_count()
processes

2

And now, let's download them with `objaverse.load_objects` with all of our processes. The function

```python
objaverse.load_objects(
    uids: List[str],
    download_processes: int = 1
) -> Dict[str, str]
```

takes in a list of object UIDs and optionally the number of download processes, and returns a map from each object UID to its `.glb` file location on disk:

In [22]:
objects = objaverse.load_objects(
    uids=filter_object_list,
    download_processes=processes
)
objects

starting download of 1 objects with 2 processes
Downloaded 1 / 1 objects


{'844ba0cf144a413ea92c779f18912042': '/root/.objaverse/hf-objaverse-v1/glbs/000-111/844ba0cf144a413ea92c779f18912042.glb'}

Let's load up one of the `.glb` files to visualize it:

In [24]:
!pip install trimesh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.1/680.1 KB 24.3 MB/s eta 0:00:00


In [25]:
import trimesh
trimesh.load(list(objects.values())[0]).show()

> **NOTE: Recommend using Blender for rendering. It will texture the meshes much better than trimesh. Here just to show a quick visualization.**

# LVIS Annotations

Finally, we'll load in the LVIS annotations, which returns a dictionary that maps from the LVIS category (key) to a list of object UIDs within that category (value):

For more information, please visit Objaverse [website](objaverse.allenai.org) or open a discussion on 🤗[Hugging Face repo](https://huggingface.co/datasets/allenai/objaverse/discussions).